In [5]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#!/usr/bin/env python
# coding: utf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

import src.features.pabellon as pb

# Configuración de gráficos
sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

ANIO_INICIO = 2017
ANIO_TERMINO = 2035
COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

In [7]:
# Cargar datos
casos_area_de_influencia = pb.cargar_casos_area_de_influencia(
    "../data/interim/casos_teoricos_diagnosticos.xlsx"
)
porcentajes_de_quirurgicos = pb.cargar_porcentajes_de_quirurgicos(
    "../data/external/incidencias_y_prevalencias_INT.xlsx"
)

# Calcular casos quirúrgicos
DIAGS_AREA_DE_INFLUENCIA = casos_area_de_influencia.index
casos_quirurgicos = pb.calcular_casos_quirurgicos(
    casos_area_de_influencia, porcentajes_de_quirurgicos
)

# Cargar y procesar duraciones de intervenciones quirúrgicas
duraciones_int_q = pb.cargar_duraciones_int_q(
    "../data/raw/5_duracion_int_q/duraciones_de_int_q_por_diagnostico.xlsx",
    DIAGS_AREA_DE_INFLUENCIA,
)

# Calcular tiempo utilizado en pabellón en horas
tiempo_utilizado_pabellon_horas = pb.calcular_tiempo_utilizado_pabellon(
    casos_quirurgicos, duraciones_int_q
)

# Calcular horas laborales
horas_laborales = pb.calcular_horas_laborales(ANIO_INICIO, ANIO_TERMINO, 12)

# Calcular cantidad de pabellones necesarios
cantidad_de_pabellones_necesarios = pb.calcular_cantidad_de_pabellones_necesarios(
    tiempo_utilizado_pabellon_horas, horas_laborales
)

Casos del área de influencia cargados y formateados:
+---+-------------+-------------------------+-------------+----------------------+-----------------+-----------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|   | Diagnostico | Diagnosticos Contenidos | Estadística | Casos (Cada 100.000) | Edad Incidencia |                                  

In [25]:
# Lee la base de GRD unida con Pabellon
grd_y_pabellon = pd.read_csv("../data/raw/5_duracion_int_q/grd_interno_y_pabellon.csv")
grd_y_pabellon["duracion"] = pd.to_timedelta(grd_y_pabellon["duracion"])

In [26]:
diagnosticos_a_reasignar = casos_area_de_influencia.dropna(subset="Diagnosticos Contenidos")
diagnosticos_a_reasignar = diagnosticos_a_reasignar["Diagnosticos Contenidos"].str.split(", ")
diagnosticos_a_reasignar = diagnosticos_a_reasignar.reset_index()

In [27]:
for row in diagnosticos_a_reasignar.itertuples():
    diagnostico_nuevo = row[1]
    diagnosticos_antiguos = row[2]
    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    grd_y_pabellon["diag_01_principal_cod"] = grd_y_pabellon["diag_01_principal_cod"].replace(
        diagnosticos_antiguos, diagnostico_nuevo
    )

Cambiando ['C341', 'C342', 'C343', 'C780', 'C782', 'D381'] a C34N
Cambiando ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q208', 'Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q218', 'Q220', 'Q221', 'Q222', 'Q223', 'Q224', 'Q225', 'Q228', 'Q230', 'Q231', 'Q233', 'Q240', 'Q241', 'Q244', 'Q245', 'Q246', 'Q248', 'Q249', 'Q250', 'Q251', 'Q253', 'Q254', 'Q255', 'Q256', 'Q257', 'Q258', 'Q259', 'Q264', 'Q268', 'Q272', 'Q273', 'Q288', 'Q289', 'Q311', 'Q320', 'Q321', 'Q330', 'Q331', 'Q332', 'Q334', 'Q338', 'Q341', 'Q348', 'Q676', 'Q677', 'Q678', 'Q765', 'Q766', 'Q767', 'Q768', 'Q769', 'Q780', 'Q790', 'Q798', 'Q839', 'Q850', 'Q858', 'Q859', 'Q874', 'Q893'] a QXXX
Cambiando ['I052', 'I080', 'I081'] a I051
Cambiando ['I352'] a I350


In [31]:
grd_y_pabellon.groupby(["diag_01_principal_cod"])["duracion"].describe()

,count,mean,std,min,25%,50%,75%,max
diag_01_principal_cod,,,,,,,,
A099,1,0 days 02:20:00,NaT,0 days 02:20:00,0 days 02:20:00,0 days 02:20:00,0 days 02:20:00,0 days 02:20:00
A150,3,0 days 01:13:20,0 days 00:11:32.820323027,0 days 01:00:00,0 days 01:10:00,0 days 01:20:00,0 days 01:20:00,0 days 01:20:00
A151,1,0 days 00:50:00,NaT,0 days 00:50:00,0 days 00:50:00,0 days 00:50:00,0 days 00:50:00,0 days 00:50:00
A152,13,0 days 02:44:13.846153846,0 days 01:20:01.321933352,0 days 00:30:00,0 days 01:50:00,0 days 02:45:00,0 days 03:20:00,0 days 05:10:00
A153,1,0 days 01:40:00,NaT,0 days 01:40:00,0 days 01:40:00,0 days 01:40:00,0 days 01:40:00,0 days 01:40:00
...,...,...,...,...,...,...,...,...
Z452,23,0 days 01:30:00,0 days 00:41:59.740246352,0 days 00:40:00,0 days 01:00:00,0 days 01:25:00,0 days 01:50:00,0 days 03:30:00
Z470,71,0 days 01:48:05.915492957,0 days 00:41:56.306577656,0 days 00:35:00,0 days 01:25:00,0 days 01:40:00,0 days 02:07:30,0 days 04:30:00
Z511,22,0 days 01:03:24.545454545,0 days 00:24:07.321754047,0 days 00:25:00,0 days 00:45:00,0 days 01:07:30,0 days 01:15:00,0 days 02:00:00
